# 다음 코드 변경
[5.7.3 ResNet.ipynb](https://github.com/silverstar0727/Pytorch_tutorial/blob/main/5%EC%9E%A5%20%ED%95%A9%EC%84%B1%EA%B3%B1%20%EC%8B%A0%EA%B2%BD%EB%A7%9D/5.7.3%20ResNet.ipynb)

## Libraries


In [1]:
import tensorflow as tf
from tensorflow.keras import *

## Hyperparameters

변경 전

In [ ]:
batch_size= 1
learning_rate = 0.0002
num_epoch = 100

변경 후

In [2]:
""" configuration json을 읽어들이는 class """
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)

In [3]:
config = Config({
    'batch_size': 1,
    'learning_rate': 0.0002,
    'num_epoch': 100
})

# Model

## 1. Basic Block

변경 전

In [ ]:
def conv_block_1(in_dim,out_dim,act_fn,stride=1):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=1, stride=stride),
        act_fn,
    )
    return model


def conv_block_3(in_dim,out_dim,act_fn):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=3, stride=1, padding=1),
        act_fn,
    )
    return model

변경 후

In [4]:

def conv_block_1(in_dim, out_dim, act_fn, stride=1):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(in_dim, out_dim, kernel_size = 1, stride = stride,
                                     activation = act_fn))
    return model


def conv_block_3(in_dim, out_dim, act_fn):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(in_dim, out_dim, kernel_size = 3, stride = 1,
                                     padding = 'same', activation = act_fn))
    return model

## 2. Bottle Neck Module

변경 전

In [ ]:
class BottleNeck(nn.Module):
    def __init__(self,in_dim,mid_dim,out_dim,act_fn,down=False):
        super(BottleNeck,self).__init__()
        self.down=down
        
        # 특성지도의 크기가 감소하는 경우
        if self.down:
            self.layer = nn.Sequential(
              conv_block_1(in_dim,mid_dim,act_fn,2),
              conv_block_3(mid_dim,mid_dim,act_fn),
              conv_block_1(mid_dim,out_dim,act_fn),
            )
            self.downsample = nn.Conv2d(in_dim,out_dim,1,2)
            
        # 특성지도의 크기가 그대로인 경우
        else:
            self.layer = nn.Sequential(
                conv_block_1(in_dim,mid_dim,act_fn),
                conv_block_3(mid_dim,mid_dim,act_fn),
                conv_block_1(mid_dim,out_dim,act_fn),
            )
            
        # 더하기를 위해 차원을 맞춰주는 부분
        self.dim_equalizer = nn.Conv2d(in_dim,out_dim,kernel_size=1)
                  
    def forward(self,x):
        if self.down:
            downsample = self.downsample(x)
            out = self.layer(x)
            out = out + downsample
        else:
            out = self.layer(x)
            if x.size() is not out.size():
                x = self.dim_equalizer(x)
            out = out + x
        return out

변경 후

In [5]:
class BottleNeck(tf.keras.Model):
    def __init__(self, in_dim, mid_dim, out_dim, act_fn, down=False):
        super(BottleNeck.self).__init__()
        self.down = down

        if self.down:
            self.layer = tf.keras.Sequential()
            self.layer.add(conv_block_1(in_dim,mid_dim,act_fn,2))
            self.layer.add(conv_block_3(mid_dim,mid_dim,act_fn))
            self.layer.add(conv_block_1(mid_dim,out_dim,act_fn))

            self.downsample = tf.keras.layers.Conv2D(in_dim, out_dim, 1, 2)

        else:
            self.layer = tf.keras.Sequential()
            self.layer.add(conv_block_1(in_dim,mid_dim,act_fn))
            self.layer.add(conv_block_3(mid_dim,mid_dim,act_fn))
            self.layer.add(conv_block_1(mid_dim,out_dim,act_fn))

        self.dim_dqualizer = tf.keras.layers.Conv2D(in_dim, out_dim, kernel_size=1)

    def call(self, x):
        if self.down:
            downsample = self.downsample(x)
            out = self.layer(x)
            out = out + downsample
        else:
            out = self.layer(x)
            if x.size() is not out.size():
                x = self.dim_dqualizer(x)
            out = out + x
        return out

## 3.ResNet Model

변경 전

In [ ]:

    def __init__(self, base_dim, num_classes=2):
        super(ResNet, self).__init__()
        self.act_fn = nn.ReLU()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,base_dim,7,2,3),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_2 = nn.Sequential(
            BottleNeck(base_dim,base_dim,base_dim*4,self.act_fn),
            BottleNeck(base_dim*4,base_dim,base_dim*4,self.act_fn),
            BottleNeck(base_dim*4,base_dim,base_dim*4,self.act_fn,down=True),
        )   
        self.layer_3 = nn.Sequential(
            BottleNeck(base_dim*4,base_dim*2,base_dim*8,self.act_fn),
            BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn),
            BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn),
            BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn,down=True),
        )
        self.layer_4 = nn.Sequential(
            BottleNeck(base_dim*8,base_dim*4,base_dim*16,self.act_fn),
            BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
            BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn),            
            BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
            BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
            BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn,down=True),
        )
        self.layer_5 = nn.Sequential(
            BottleNeck(base_dim*16,base_dim*8,base_dim*32,self.act_fn),
            BottleNeck(base_dim*32,base_dim*8,base_dim*32,self.act_fn),
            BottleNeck(base_dim*32,base_dim*8,base_dim*32,self.act_fn),
        )
        self.avgpool = nn.AvgPool2d(7,1) 
        self.fc_layer = nn.Linear(base_dim*32,num_classes)
        
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out = self.avgpool(out)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        
        return out

변경 후

In [6]:
class ResNet(tf.keras.Model):

    def __init__(self, base_dim, num_classes = 2):
        super(ResNet, self).__init__()
        self.act_fn = 'relu'

        self.layer_1 = tf.keras.Sequential()
        self.layer_1.add(tf.keras.layers.Conv2D(filters=base_dim, kernel_size=7,
                                                stride=2, padding=3, activation='relu'))
        self.layer_1.add(tf.keras.layers.MaxPool2D((2,2)))

        self.layer_2 = tf.keras.Sequential()
        self.layer_2.add(BottleNeck(base_dim,base_dim,base_dim*4,self.act_fn))
        self.layer_2.add(BottleNeck(base_dim*4,base_dim,base_dim*4,self.act_fn))
        self.layer_2.add(BottleNeck(base_dim*4,base_dim,base_dim*4,self.act_fn,down=True))

        self.layer_3 = tf.keras.Sequential()
        self.layer_3.add(BottleNeck(base_dim*4,base_dim*2,base_dim*8,self.act_fn))
        self.layer_3.add(BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn))
        self.layer_3.add(BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn))
        self.layer_3.add(BottleNeck(base_dim*8,base_dim*2,base_dim*8,self.act_fn,down=True))

        self.layer_4 = tf.keras.Sequential()
        self.layer_4.add(BottleNeck(base_dim*8,base_dim*4,base_dim*16,self.act_fn))
        self.layer_4.add(BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn))
        self.layer_4.add(BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn))
        self.layer_4.add(BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn))
        self.layer_4.add(BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn))
        self.layer_4.add(BottleNeck(base_dim*16,base_dim*4,base_dim*16,self.act_fn,down=True))

        self.layer_5 = tf.keras.Sequential()
        self.layer_5.add(BottleNeck(base_dim*16,base_dim*8,base_dim*32,self.act_fn))
        self.layer_5.add(BottleNeck(base_dim*32,base_dim*8,base_dim*32,self.act_fn))
        self.layer_5.add(BottleNeck(base_dim*32,base_dim*8,base_dim*32,self.act_fn))

        self.avgpool = tf.keras.layers.AveragePooling2D(pool_size = 7, strides = 1)
        self.fc_layer = tf.keras.layers.Dense(num_classes)

    def call(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out = self.avgpool(out)
        out = tf.reshape(out, (self.config.batch_size, -1))
        out = self.fc_layer(out)
        return out
